In [11]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset
#from torchvision.datasets import MNIST
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
#import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn import preprocessing

#from torchvision.transforms import ToTensor
#from torchvision.utils import make_grid
from torch.utils.data import random_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from torch import Tensor
from torch.nn.parameter import Parameter
import torch.nn.init
from torch.nn.modules import Module
import math

from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
# Use a white background for matplotlib figures
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

import warnings
warnings.filterwarnings("ignore")
import openml
import random

In [12]:
def calculate_metrics(outputs, labels):    
    labels_onehot = torch.nn.functional.one_hot(labels, num_classes=2).cpu().detach().numpy()
    _, preds = torch.max(outputs, dim=1)
    #print(outputs.shape)
    outputs = outputs.cpu().detach().numpy()
    outputs = outputs.reshape(outputs.shape[0]*outputs.shape[1],outputs.shape[2])
    labels_onehot = labels_onehot.reshape(labels_onehot.shape[0]*labels_onehot.shape[1],labels_onehot.shape[2])
    #acc = torch.tensor(torch.sum(preds == labels).item() / len(preds)).mean()
    auc = np.round(roc_auc_score(labels_onehot,outputs),4)
    mcc=0
    f=0
    #mcc = matthews_corrcoef(labels,preds)
    #f = f1_score(labels,preds, average='weighted')
    return auc, mcc, f

#Define accuracy metric
def accuracy(outputs, labels):
    labels_onehot = torch.nn.functional.one_hot(labels, num_classes=10).cpu().detach().numpy()
    _, preds = torch.max(outputs, dim=1)
    acc = torch.tensor(np.round(torch.sum(preds == labels).item() / len(preds),4))
    #global_outputs.append(outputs.detach().numpy())
    return acc

In [13]:
class HyperNet(nn.Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight1: Tensor
    bias1: Tensor
    bias2: Tensor
    weight2: Tensor
    zval: Tensor
        
    def __init__(self, in_features: int, out_features: int, z_dim: int, bias: bool = False,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(HyperNet, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.z_dim = z_dim
        self.weight1 = nn.Parameter(torch.fmod(torch.randn((in_features,z_dim),**factory_kwargs),2))
        self.weight2 = nn.Parameter(torch.fmod(torch.randn((z_dim,out_features),**factory_kwargs),2))
        if bias:
            self.bias1 = nn.Parameter(torch.fmod(torch.randn((z_dim), **factory_kwargs),2))
            self.bias2 = nn.Parameter(torch.fmod(torch.randn((out_features), **factory_kwargs),2))
        else:
            self.register_parameter('bias', None)
        
    def forward(self, xb):
        xb = torch.matmul(xb,self.weight1) + self.bias1
        # NO activation required
        # output of hypernet layer2
        xb = torch.matmul(xb,self.weight2) + self.bias2
        #return weights for main network
        return xb

In [14]:
#Define model
class MnistModel(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, z_dim):
        super().__init__()
        # hidden layer
        self.linear1 = HyperNet(in_size, hidden_size, z_dim, bias=True)
        # output layer
        self.linear2 = HyperNet(hidden_size, out_size, z_dim, bias=True)
        
    def forward(self, xb):
        # Flatten the image tensors
        ##xb = (xb.view(xb.size(0), -1)).clone().detach()
        # Get intermediate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        out = F.tanh(out)
        # Get predictions using output layer
        out = self.linear2(out)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)
        # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'loss': loss, 'acc': acc, 'out': out, 'labels': labels}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        batch_out = [x['out'] for x in outputs]
        epoch_out = torch.stack(batch_out)
        batch_labels = [x['labels'] for x in outputs]
        epoch_labels = torch.stack(batch_labels)
        epoch_auc, mcc, f = calculate_metrics(epoch_out, epoch_labels)
        return {'epoch_loss': epoch_loss.item(), 'epoch_acc': epoch_acc.item(), 'epoch_auc': epoch_auc.item()}
    
    def epoch_end(self, epoch, result):
        return
        #print("Epoch [{}], loss: {:.4f}, acc: {:.4f}, auc score: {:.4f}".format(epoch, result['epoch_loss'],result['epoch_acc'],result['epoch_auc']))

In [15]:
#split training and validation indices
def split_indices(n, val_pct):
    n_val = int(val_pct*n)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

In [16]:
#Move to cuda if available
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    else:
        return data.to(device, non_blocking=True)
    
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
            
    def __len__(self):
        return len(self.dl)

In [17]:
#Define fit function
def evaluate(model, loader):
    """Evaluate the model's performance on the validation set"""
    outputs=[]
    for batch in loader:
        outputs.append(model.validation_step(batch))
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, test_loader, opt_func=torch.optim.SGD):
    """Train the model using gradient descent"""
    history = []
    last_loss=0
    error_queue = []
    avg_error=1
    last_iter=0
    #loss_threshold=0.37
    row['Learning Rate'] = lr
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        last_iter = epoch + 1
        total_loss=0
        num_batches=0
        for batch in train_loader:
            loss = model.training_step(batch)
            
            total_loss = total_loss + loss
            num_batches = num_batches+1
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        avg_loss = total_loss/num_batches
        change = abs(last_loss - avg_loss)
        #if loss < loss_threshold:
        if len(error_queue) < 20:
            error_queue.append(abs(change))
            #print(error_queue)
        else:
            error_queue.append(abs(change))
            #print(error_queue)
            avg_error = sum(error_queue[1:21])/len(error_queue[1:21])
            error_queue.pop(0)

            #print(avg_error)       
            if avg_error <= 0.00000001: #Error threshold based early stopping
                break

        last_loss = loss
            
    row["Iter/Generation"] = last_iter
    # Training phase
    result = evaluate(model, train_loader)
    model.epoch_end(epochs, result)
    history.append(result.copy())
    
    # Validation phase
    result = evaluate(model, val_loader)
    model.epoch_end(epochs, result)
    history.append(result.copy())
    
    # Testing Phase
    result = evaluate(model, test_loader)
    model.epoch_end(epochs, result)
    history.append(result.copy())
    
    return history

In [22]:
class CustomDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        #df = pd.read_csv(annotations_file)
        df = annotations_file
        self.labels = df['label']
        input_size = len(df.columns)-1
        self.data = df.iloc[:,0:input_size]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.data.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = np.round(min_max_scaler.fit_transform(x), 3)
        self.data = pd.DataFrame(x_scaled)
        
        label_read = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        data_read = torch.tensor(self.data.iloc[idx], dtype=torch.float32)
        return data_read, label_read


def get_data(Dataset):
    batch_size=25
    device = get_default_device()
    #num_classes=2
    dataset = CustomDataset(Dataset)
    train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6,0.2,0.2], generator=torch.Generator())
    train_loader = DataLoader(train_dataset, batch_size,drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size,drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size,drop_last=True)
    
    train_loader = DeviceDataLoader(train_loader, device)
    val_loader = DeviceDataLoader(val_loader, device)
    test_loader = DeviceDataLoader(test_loader, device)
    return train_loader, val_loader, test_loader

In [19]:
'''
def get_data(Dataset):
    batch_size=25
    '''Date Input/Output code for CSV dataset'''
   '''
    device = get_default_device()
    #df = pd.read_csv('../Dataset/pneumonia/pneumonia_train.csv')
    df = pd.read_csv('../Dataset/Wisconsin_Preprocessed.csv')
    #print(df.shape)
    
    num_classes=2
    # Reading the input data.
    dataset = torch.Tensor(np.array(df))
    input_size = len(df.columns)-1
    df = df.iloc[:,0:input_size]
    dataset = CustomDataset("../Dataset/Wisconsin_Preprocessed.csv")
    train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6,0.2,0.2], generator=torch.Generator())
    train_loader = DataLoader(train_dataset, batch_size,drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size,drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size,drop_last=True)
    
    train_loader = DeviceDataLoader(train_loader, device)
    val_loader = DeviceDataLoader(val_loader, device)
    test_loader = DeviceDataLoader(test_loader, device)
    
    return train_loader, val_loader, test_loader

SyntaxError: invalid syntax (1173698572.py, line 4)

In [20]:
import os
def save_to_csv(df, name):
    path = 'Output/' + name
    if not os.path.exists('Output'):
        os.makedirs('Output')
    df.to_csv(path, index=False)

def create_csv():
    columns = ['Dataset', 'Instances','Features', 'Hidden Layer', 'Output Layer',
               'Iter/Generation', 'Input Size', 
               'Learning Rate', 'Train Accuracy',
               'Train AUC', 'Val Accuracy',
               'Val AUC', 'Test Accuracy',
               'Test AUC', 'Time Taken(s)',
               'Time per generation', 'Memory Required', 'Total Memory(kB)',
               'Algorithmic Complexity']
    
    return pd.DataFrame(columns=columns)

In [ ]:
import time
from sklearn.preprocessing import LabelEncoder

def run(layer, train_loader, val_loader, test_loader):
    global df, input_size
    #print(hidden_size)
    z_dim=hidden_size/2
    start = time.time()
    #result = [{'epoch_acc': 1, 'epoch_auc': 2}, {'epoch_acc': 1, 'epoch_auc': 2}, {'epoch_acc': 1, 'epoch_auc': 2}]
    result = fit(500, 0.01, model, train_loader, val_loader, test_loader)
    row['Time Taken(s)'] = time.time() - start
    row['Time per generation'] = row['Time Taken(s)'] / row['Iter/Generation']
    row['Input Size'] = input_size
    #row['Num Classes'] = num_classes
    #row['Z Dim'] = z_dim

    row['Train Accuracy'] = result[0]['epoch_acc']
    row['Train AUC'] = result[0]['epoch_auc']
    row['Val Accuracy'] = result[1]['epoch_acc']
    row['Val AUC'] = result[1]['epoch_auc']
    row['Test Accuracy'] = result[2]['epoch_acc']
    row['Test AUC'] = result[2]['epoch_auc']
    
    memory_req = (input_size + hidden_size + num_classes) * z_dim
    row['Memory Required'] = memory_req
    row['Total Memory(kB)'] = memory_req/1024.0
    row['Algorithmic Complexity'] = (memory_req * row['Iter/Generation'])/10000.0
    row['Output Layer'] = num_classes
    row['Hidden Layer'] = layer

    df1 = pd.DataFrame(row, index=[0])
    df = pd.concat([df, df1], ignore_index=True)

df = create_csv()
if __name__ == '__main__':
    input_size = 0
    num_classes = 0
    repeat = 5
    layers = [10, 20, 50, 100, 200]
    #layers= [10] 472, 946 736,782,
    #small datasets 1
    #dataset_final = [714]
    #dataset_final = [924,885,974,784,902,969,955,748,801,446,733,796,996,1005,895,464,450,1073,880,1121,44149,40710,43,818,1524,915,1167,925,1011,1048,860,900,906,907,908,909,1511,1498,724,814,4329,750,886,987,825,717,1063,1467,947,949,950,951,41945,770,997,774,795,827,931,40981,1452,1464,37,994,841,40705,1016,31,1547,43255,741,1444,1494,45077,1453,1068,1462,1049,983,1050,1504]
    #small datasets 2         - run in server - 1116, 40666,
    dataset_final = [803,1496,1507,43946,725,735,752,761,807,816,833,923,1056,819,45553,976]
    #dataset_final = [962,971,995,1020,914,1067,772,40704,958,40999,41007,316,1487,43980,45562,41143,737,41005,40713,41946,871,728,41156,720,44,40983,43892,43949,979,40701,43947,40900,41146,43958,42192,1460,1489,1021,1069,980,847,1116,40666,803,1496,1507,43946,725,735,752,761,807,816,833,923,1056,819,45553,976]
    #dataset_final = 1504,1507,1511,1524,1547,1597,4154,4329,23517,40666,40701,40704,40705,40710,40713,40900,40922,40981,40983,40999,41005,41007,41143,41146,41150,41156,41228,41945,41946,42192,42397,42477,42680,42769,43255,43551,43890,43892,43946,43947,43949,43958,43977,43979,43980,44038,44089,44149,44162,45023,45035,45037,45060,45077,45547,45549,45553,45556,45558,45562,45578]
    #dataset_final = [31,37,43,44,151,161,162,246,251,257,260,264,267,269,310,316,446,450,464,472,714,717,720,722,724,725,728,733,734,735,736,737,741,748,750,752,761,770,772,774,782,784,795,796,801,803,807,814,816,818,819,821,823,825,827,833,841,843,846,847,860,871,880,881,885,886,895,900,901,902,906,907,908,909,914,915,923,924,925,931,946,947,949,950,951,955,958,962,969,971,974,976,977,979,980,983,987,994,995,996,997,1005,1011,1016,1019,1020,1021,1040,1046,1048,1049,1050,1056,1063,1067,1068,1069,1073,1116,1120,1121,1167,1169,1182,1205,1212,1216,1444,1452,1453,1460,1461,1462,1464,1467,1471,1486,1487,1489,1494,1496,1498,1502,]
    #hidden_size = layers

    for did in dataset_final:
        row = dict()    
        row['Dataset'] = did
        print(did)
        dataset = openml.datasets.get_dataset(did)
    
        #get data from the dataset
        X, y, categorical_indicator, attribute_names = dataset.get_data(
            dataset_format="dataframe", target=dataset.default_target_attribute
        )
        
        print(categorical_indicator)
        i_list=[]
        i=0
        for ci in categorical_indicator:
            if ci == False:
                i_list.append(i)
            i=i+1
        print(i_list)
    
        X=X.iloc[:,i_list].astype("float")

        le = LabelEncoder()
        y = le.fit_transform(y)
        num_features = len(X.T)
        X['label']=y.T
        print("features="+str(num_features))
        max_samples = int(len(X))
        if max_samples >10000:
            max_samples = 10000
            X.sample(n=max_samples, random_state=1)
        row['Instances'] = max_samples
        row['Features'] = num_features
        for rep in range(repeat):       
            print("rep=",rep)
            train_loader, val_loader, test_loader = get_data(X)
            for layer in layers:
                for batch in train_loader:
                    data, labels = batch
                    input_size = len(data.T)
                    break
                num_classes = 2
                hidden_size=layer
                model = MnistModel(input_size, hidden_size=layer, out_size=num_classes, z_dim=5)
                model = model.to(device=get_default_device())
                run(layer, train_loader, val_loader, test_loader)
            
            #display(df)
            
save_to_csv(df, f'HN_{time.time()}.txt')

803
[False, False, False, False, False]
[0, 1, 2, 3, 4]
features=5
rep= 0
rep= 1
rep= 2
rep= 3
rep= 4
1496
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
features=20
rep= 0
rep= 1
rep= 2
rep= 3
rep= 4
1507
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
features=20
rep= 0
rep= 1
rep= 2


In [ ]:
display(df)

In [1]:
save_to_csv(df, f'HN_{time.time()}.txt')

NameError: name 'save_to_csv' is not defined